First mount Google drive for later use


Import all dependencies, first making sure that opencv, matplotlib, and numpy are installed via pip

In [1]:
import cv2
import matplotlib
import numpy as np
from google.colab.patches import cv2_imshow
from itertools import combinations_with_replacement
from collections import defaultdict
from numpy.linalg import inv

Image Enhancement; run block to define all enhancement functions

In [3]:
def get_illumination_channel(I, w):
    M, N, _ = I.shape
    # padding for channels
    padded = np.pad(I, ((int(w/2), int(w/2)), (int(w/2), int(w/2)), (0, 0)), 'edge')
    darkch = np.zeros((M, N))
    brightch = np.zeros((M, N))

    for i, j in np.ndindex(darkch.shape):
        darkch[i, j] = np.min(padded[i:i + w, j:j + w, :]) # dark channel
        brightch[i, j] = np.max(padded[i:i + w, j:j + w, :]) # bright channel

    return darkch, brightch

def get_atmosphere(I, brightch, p=0.1):
    M, N = brightch.shape
    flatI = I.reshape(M*N, 3) # reshaping image array
    flatbright = brightch.ravel() #flattening image array

    searchidx = (-flatbright).argsort()[:int(M*N*p)] # sorting and slicing
    A = np.mean(flatI.take(searchidx, axis=0), dtype=np.float64, axis=0)
    return A

def get_initial_transmission(A, brightch):
    A_c = np.max(A)
    init_t = (brightch-A_c)/(1.-A_c) # finding initial transmission map
    return (init_t - np.min(init_t))/(np.max(init_t) - np.min(init_t)) # normalized initial transmission map

def get_corrected_transmission(I, A, darkch, brightch, init_t, alpha, omega, w):
    im = np.empty(I.shape, I.dtype);
    for ind in range(0, 3):
        im[:, :, ind] = I[:, :, ind] / A[ind] #divide pixel values by atmospheric light
    dark_c, _ = get_illumination_channel(im, w) # dark channel transmission map
    dark_t = 1 - omega*dark_c # corrected dark transmission map
    corrected_t = init_t # initializing corrected transmission map with initial transmission map
    diffch = brightch - darkch # difference between transmission maps

    for i in range(diffch.shape[0]):
        for j in range(diffch.shape[1]):
            if(diffch[i, j] < alpha):
                corrected_t[i, j] = dark_t[i, j] * init_t[i, j]

    return np.abs(corrected_t)

R, G, B = 0, 1, 2  # index for convenience

def boxfilter(I, r):
    """Fast box filter implementation.

    Parameters
    ----------
    I:  a single channel/gray image data normalized to [0.0, 1.0]
    r:  window radius

    Return
    -----------
    The filtered image data.
    """
    M, N = I.shape
    dest = np.zeros((M, N))
    #print(I)

    # cumulative sum over Y axis (tate-houkou no wa)
    sumY = np.cumsum(I, axis=0)
    #print('sumY:{}'.format(sumY))
    # difference over Y axis
    dest[:r + 1] = sumY[r:2*r + 1] # top r+1 lines
    dest[r + 1:M - r] = sumY[2*r + 1:] - sumY[:M - 2*r - 1]
    #print(sumY[2*r + 1:]) # from 2*r+1 to end lines
    #print(sumY[:M - 2*r - 1]) # same lines of above, from start
    #tile replicate sumY[-1] and line them up to match the shape of (r, 1)
    dest[-r:] = np.tile(sumY[-1], (r, 1)) - sumY[M - 2*r - 1:M - r - 1] # bottom r lines

    # cumulative sum over X axis
    sumX = np.cumsum(dest, axis=1)
    #print('sumX:{}'.format(sumX))
    # difference over X axis
    dest[:, :r + 1] = sumX[:, r:2*r + 1] # left r+1 columns
    dest[:, r + 1:N - r] = sumX[:, 2*r + 1:] - sumX[:, :N - 2*r - 1]
    dest[:, -r:] = np.tile(sumX[:, -1][:, None], (1, r)) - sumX[:, N - 2*r - 1:N - r - 1] # right r columns

    #print(dest)

    return dest

def guided_filter(I, p, r=15, eps=1e-3):
    """Refine a filter under the guidance of another (RGB) image.

    Parameters
    -----------
    I:   an M * N * 3 RGB image for guidance.
    p:   the M * N filter to be guided. transmission is used for this case.
    r:   the radius of the guidance
    eps: epsilon for the guided filter

    Return
    -----------
    The guided filter.
    """
    M, N = p.shape
    base = boxfilter(np.ones((M, N)), r) # this is needed for regularization

    # each channel of I filtered with the mean filter. this is myu.
    means = [boxfilter(I[:, :, i], r) / base for i in range(3)]

    # p filtered with the mean filter
    mean_p = boxfilter(p, r) / base

    # filter I with p then filter it with the mean filter
    means_IP = [boxfilter(I[:, :, i]*p, r) / base for i in range(3)]

    # covariance of (I, p) in each local patch
    covIP = [means_IP[i] - means[i]*mean_p for i in range(3)]

    # variance of I in each local patch: the matrix Sigma in ECCV10 eq.14
    var = defaultdict(dict)
    for i, j in combinations_with_replacement(range(3), 2):
        var[i][j] = boxfilter(I[:, :, i]*I[:, :, j], r) / base - means[i]*means[j]

    a = np.zeros((M, N, 3))
    for y, x in np.ndindex(M, N):
        #         rr, rg, rb
        # Sigma = rg, gg, gb
        #         rb, gb, bb
        Sigma = np.array([[var[R][R][y, x], var[R][G][y, x], var[R][B][y, x]],
                          [var[R][G][y, x], var[G][G][y, x], var[G][B][y, x]],
                          [var[R][B][y, x], var[G][B][y, x], var[B][B][y, x]]])
        cov = np.array([c[y, x] for c in covIP])
        a[y, x] = np.dot(cov, inv(Sigma + eps*np.eye(3)))  # eq 14

    # ECCV10 eq.15
    b = mean_p - a[:, :, R]*means[R] - a[:, :, G]*means[G] - a[:, :, B]*means[B]

    # ECCV10 eq.16
    q = (boxfilter(a[:, :, R], r)*I[:, :, R] + boxfilter(a[:, :, G], r)*I[:, :, G] + boxfilter(a[:, :, B], r)*I[:, :, B] + boxfilter(b, r)) / base

    return q

def get_final_image(I, A, refined_t, tmin):
    refined_t_broadcasted = np.broadcast_to(refined_t[:, :, None], (refined_t.shape[0], refined_t.shape[1], 3)) # duplicating the channel of 2D refined map to 3 channels
    J = (I-A) / (np.where(refined_t_broadcasted < tmin, tmin, refined_t_broadcasted)) + A # finding result

    return (J - np.min(J))/(np.max(J) - np.min(J)) # normalized image

def reduce_init_t(init_t):
    init_t = (init_t*255).astype(np.uint8)
    xp = [0, 32, 255]
    fp = [0, 32, 48]
    x = np.arange(256) # creating array [0,...,255]
    table = np.interp(x, xp, fp).astype('uint8') # interpreting fp according to xp in range of x
    init_t = cv2.LUT(init_t, table) # lookup table
    init_t = init_t.astype(np.float64)/255 # normalizing the transmission map
    return init_t

def dehaze(I, tmin=0.1, w=15, alpha=0.4, omega=0.75, p=0.1, eps=1e-3, reduce=False):
    I = np.asarray(I, dtype=np.float64) # Convert the input to a float array.
    I = I[:, :, :3] / 255
    m, n, _ = I.shape
    Idark, Ibright = get_illumination_channel(I, w)
    A = get_atmosphere(I, Ibright, p)

    init_t = get_initial_transmission(A, Ibright)
    if reduce:
        init_t = reduce_init_t(init_t)
    corrected_t = get_corrected_transmission(I, A, Idark, Ibright, init_t, alpha, omega, w)

    normI = (I - I.min()) / (I.max() - I.min())
    refined_t = guided_filter(normI, corrected_t, w, eps) # applying guided filter
    J_refined = get_final_image(I, A, refined_t, tmin)

    enhanced = (J_refined*255).astype(np.uint8)
    f_enhanced = cv2.detailEnhance(enhanced, sigma_s=10, sigma_r=0.15)
    f_enhanced = cv2.edgePreservingFilter(f_enhanced, flags=1, sigma_s=64, sigma_r=0.2)
    return f_enhanced

 Download the supplied 'Night_Video.mov" file into content/drive/MyDrive/CS362V/data/

Video Capture and Object Tracking using background subtraction; run the block and the program will process the video and display the object detection on the video both before enhancement and after as a side-by-side comparison for each frame


In [ ]:
# Path to video
cap = cv2.VideoCapture("/content/drive/MyDrive/CS362V/data/Night_Video.mov")

# Object detection from Stable camera
object_detector = cv2.createBackgroundSubtractorMOG2(varThreshold=20)

# List to hold memory of previous frame for still object detection
prev_store = []

# Size of the first frame
baseline_size = 0

for i in range(80, 630, 12):
    # Select the frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()

    if frame is None:
      break

    key = cv2.waitKey(30)
    if key == 27:
      break

    # Create an enhanced version of the frame as a copy
    enhanced_frame = dehaze(frame)

    # Check to see if first frame size has already been measured
    if not baseline_size:
      # Measure it if not
      baseline_size = enhanced_frame.size

    # Object detection on both the regular frame and the enhanced version
    mask = object_detector.apply(frame)
    e_mask = object_detector.apply(enhanced_frame)

    # Threshold both versions
    _, mask_thresh = cv2.threshold(mask, 180, 255, cv2.THRESH_BINARY)
    _, e_mask_thresh = cv2.threshold(e_mask, 180, 255, cv2.THRESH_BINARY)

    # Kernel for erosion
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

    # Erode both versions
    mask_eroded = cv2.morphologyEx(mask_thresh, cv2.MORPH_OPEN, kernel)
    e_mask_eroded = cv2.morphologyEx(e_mask_thresh, cv2.MORPH_OPEN, kernel)

    # Find contours in both versions
    contours, _ = cv2.findContours(mask_eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    e_contours, _ = cv2.findContours(e_mask_eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Keep only the contours that contain large objects for both versions
    min_contour_area = 500
    large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]
    e_large_contours = [cnt for cnt in e_contours if cv2.contourArea(cnt) > min_contour_area]

    # Create a copy of both versions
    frame_out = frame.copy()
    e_frame_out = enhanced_frame.copy()

    # List to track objects that were moving but stopped
    stationary_rects = []

    # Checks if previous frame detected moving objects that slowed down or stopped
    if len(prev_store):
      for prev in prev_store:
        # Get ratio of roi from previous frame size to the baseline size
        roi_to_baseline = baseline_size / prev[0].size

        # Create a grayed out roi copy of the current frame using the coordinates
        # of the roi passed in from the previous frame's detections
        gray_curr_roi = cv2.cvtColor(enhanced_frame[prev[1][1]:prev[1][3],
                                               prev[1][0]:prev[1][2]], cv2.COLOR_BGR2GRAY)

        # Create a histogram of the grayed roi
        curr_histogram = cv2.calcHist([gray_curr_roi], [0],
                         None, [256], [0, 256])

        # Create a grayed out copy of the previous frame's roi
        gray_prev_roi = cv2.cvtColor(prev[0], cv2.COLOR_BGR2GRAY)

        # Histogram of the grayed out previous roi
        prev_histogram = cv2.calcHist([gray_prev_roi], [0],
                         None, [256], [0, 256])

        # Counter to track 'difference score' between current roi and previous roi
        c1 = 0

        # Loop through each pixel in each roi and multiply the differnce in
        # histogram values by the roi to baseline ratio to normalize values
        i = 0
        while i<len(curr_histogram) and i<len(prev_histogram):
            c1+=abs((curr_histogram[i]-prev_histogram[i]) * roi_to_baseline)
            i+= 1

        # Border the roi in the current frame if the difference score is
        # less than 100000, which appears to be the range that non-motion
        # is calculated at
        if c1 < 100000:
          e_frame_out = cv2.rectangle(e_frame_out, (prev[1][0], prev[1][1]), (prev[1][2], prev[1][3]), (0, 255, 0), 2)
          stationary_rects.append(prev[1])

    # Clear previous frame memory to prepare for remembering current frame
    prev_store = []

    # Store bounding rects for each contour that was large enough
    rects = []
    rectsUsed = []
    for cnt in large_contours:
        rects.append(cv2.boundingRect(cnt))
        rectsUsed.append(False)

    # Same for enhanced version
    e_rects = []
    e_rects_used = []
    for cnt in e_large_contours:
        e_rects.append(cv2.boundingRect(cnt))
        e_rects_used.append(False)

    # Small helper function that returns the x value
    def getXFromRect(item):
      return item[0]

    # Uses helper function to sort by x value
    rects.sort(key = getXFromRect)

    # List of rectangles that are going to be drawn at the end
    acceptedRects = []

    # Maximum horizontal distance that two bounding boxes can be in order
    # for them to be joined together into one large box
    xThr = 50

    # Loop through each bounding rect and check if it can be joined with others
    for supIdx, supVal in enumerate(rects):
      if (rectsUsed[supIdx] == False):

          # Initialize current rect
          currxMin = supVal[0]
          currxMax = supVal[0] + supVal[2]
          curryMin = supVal[1]
          curryMax = supVal[1] + supVal[3]

          # This bounding rect is used
          rectsUsed[supIdx] = True

          # Iterate all initial bounding rects
          # starting from the next
          for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

              # Initialize merge candidate
              candxMin = subVal[0]
              candxMax = subVal[0] + subVal[2]
              candyMin = subVal[1]
              candyMax = subVal[1] + subVal[3]

              # Check if x distance between current rect
              # and merge candidate is small enough
              if (candxMin <= currxMax + xThr):

                  # Reset coordinates of current rect
                  currxMax = candxMax
                  curryMin = min(curryMin, candyMin)
                  curryMax = max(curryMax, candyMax)

                  # Merge candidate (bounding rect) is used
                  rectsUsed[subIdx] = True
              else:
                  break

          # No more merge candidates possible, accept current rect
          acceptedRects.append([currxMin, curryMin, currxMax - currxMin, curryMax - curryMin])

    # Draw bounding boxes for regular frame
    for rect in acceptedRects:
      img = cv2.rectangle(frame_out, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (121, 11, 189), 2)

    # Repeat the actions for bounding boxes above for enhanced version
    e_rects.sort(key = getXFromRect)

    e_acceptedRects = []

    xThr = 50

    for supIdx, supVal in enumerate(e_rects):
      if (e_rects_used[supIdx] == False):

          # Initialize current rect
          currxMin = supVal[0]
          currxMax = supVal[0] + supVal[2]
          curryMin = supVal[1]
          curryMax = supVal[1] + supVal[3]

          # This bounding rect is used
          e_rects_used[supIdx] = True

          # Iterate all initial bounding rects
          # starting from the next
          for subIdx, subVal in enumerate(e_rects[(supIdx+1):], start = (supIdx+1)):

              # Initialize merge candidate
              candxMin = subVal[0]
              candxMax = subVal[0] + subVal[2]
              candyMin = subVal[1]
              candyMax = subVal[1] + subVal[3]

              # Check if x distance between current rect
              # and merge candidate is small enough
              if (candxMin <= currxMax + xThr):

                  # Reset coordinates of current rect
                  currxMax = candxMax
                  curryMin = min(curryMin, candyMin)
                  curryMax = max(curryMax, candyMax)

                  # Merge candidate (bounding rect) is used
                  e_rects_used[subIdx] = True
              else:
                  break

          # No more merge candidates possible, accept current rect
          e_acceptedRects.append([currxMin, curryMin, currxMax - currxMin, curryMax - curryMin])

    # Check to see if any of the accepted bounding boxes overlap with one of the
    # stationary boxes already drawn. If so, skip drawing that bounding box
    for rect in e_acceptedRects:
      x1 = rect[0]
      x2 = rect[2] + rect[0]
      y1 = rect[1]
      y2 = rect[3] + rect[1]

      valid = True
      # Loop through stationary rects
      if len(stationary_rects):
        for s_rect in stationary_rects:
          # Check for overlap
          if s_rect[0] <= rect[0] <= s_rect[2] or s_rect[0] <= rect[2] <= s_rect[2] or s_rect[1] <= rect[1] <= s_rect[3] or s_rect[1] <= rect[3] <= s_rect[3]:
            valid = False
            break
        if valid:
          # Draw box if no overlaps
          e_frame_out = cv2.rectangle(e_frame_out, (x1, y1), (x2, y2), (121, 11, 189), 2)
      else:
          # Draw box if there were no stationary boxes
        e_frame_out = cv2.rectangle(e_frame_out, (x1, y1), (x2, y2), (121, 11, 189), 2)

     # Add all the detections to the previous frame memory
      prev_store.append((enhanced_frame[y1:y2, x1:x2], [x1, y1, x2, y2]))

    # Join the regualar and enhanced frames to see side by side comparison
    joint = np.hstack([frame_out, e_frame_out])
    cv2_imshow(joint)



cap.release()

Output hidden; open in https://colab.research.google.com to view.

Same thing but using the video with stops to check stationary object detection. A green box around an object means it was previously moving but now stopped

In [5]:
# Path to video
cap = cv2.VideoCapture("/content/drive/MyDrive/CS362V/data/Night_Video_With_Stops.mov")

# Object detection from Stable camera
object_detector = cv2.createBackgroundSubtractorMOG2(varThreshold=20)

# List to hold memory of previous frame for still object detection
prev_store = []

# Size of the first frame
baseline_size = 0

for i in range(80, 1000, 12):
    # Select the frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()

    if frame is None:
      break

    key = cv2.waitKey(30)
    if key == 27:
      break

    # Create an enhanced version of the frame as a copy
    enhanced_frame = dehaze(frame)

    # Check to see if first frame size has already been measured
    if not baseline_size:
      # Measure it if not
      baseline_size = enhanced_frame.size

    # Object detection on both the regular frame and the enhanced version
    mask = object_detector.apply(frame)
    e_mask = object_detector.apply(enhanced_frame)

    # Threshold both versions
    _, mask_thresh = cv2.threshold(mask, 180, 255, cv2.THRESH_BINARY)
    _, e_mask_thresh = cv2.threshold(e_mask, 180, 255, cv2.THRESH_BINARY)

    # Kernel for erosion
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

    # Erode both versions
    mask_eroded = cv2.morphologyEx(mask_thresh, cv2.MORPH_OPEN, kernel)
    e_mask_eroded = cv2.morphologyEx(e_mask_thresh, cv2.MORPH_OPEN, kernel)

    # Find contours in both versions
    contours, _ = cv2.findContours(mask_eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    e_contours, _ = cv2.findContours(e_mask_eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Keep only the contours that contain large objects for both versions
    min_contour_area = 500
    large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]
    e_large_contours = [cnt for cnt in e_contours if cv2.contourArea(cnt) > min_contour_area]

    # Create a copy of both versions
    frame_out = frame.copy()
    e_frame_out = enhanced_frame.copy()

    # List to track objects that were moving but stopped
    stationary_rects = []

    # Checks if previous frame detected moving objects that slowed down or stopped
    if len(prev_store):
      for prev in prev_store:
        # Get ratio of roi from previous frame size to the baseline size
        roi_to_baseline = baseline_size / prev[0].size

        # Create a grayed out roi copy of the current frame using the coordinates
        # of the roi passed in from the previous frame's detections
        gray_curr_roi = cv2.cvtColor(enhanced_frame[prev[1][1]:prev[1][3],
                                               prev[1][0]:prev[1][2]], cv2.COLOR_BGR2GRAY)

        # Create a histogram of the grayed roi
        curr_histogram = cv2.calcHist([gray_curr_roi], [0],
                         None, [256], [0, 256])

        # Create a grayed out copy of the previous frame's roi
        gray_prev_roi = cv2.cvtColor(prev[0], cv2.COLOR_BGR2GRAY)

        # Histogram of the grayed out previous roi
        prev_histogram = cv2.calcHist([gray_prev_roi], [0],
                         None, [256], [0, 256])

        # Counter to track 'difference score' between current roi and previous roi
        c1 = 0

        # Loop through each pixel in each roi and multiply the differnce in
        # histogram values by the roi to baseline ratio to normalize values
        i = 0
        while i<len(curr_histogram) and i<len(prev_histogram):
            c1+=abs((curr_histogram[i]-prev_histogram[i]) * roi_to_baseline)
            i+= 1

        # Border the roi in the current frame if the difference score is
        # less than 100000, which appears to be the range that non-motion
        # is calculated at
        if c1 < 100000:
          e_frame_out = cv2.rectangle(e_frame_out, (prev[1][0], prev[1][1]), (prev[1][2], prev[1][3]), (0, 255, 0), 2)
          stationary_rects.append(prev[1])

    # Clear previous frame memory to prepare for remembering current frame
    prev_store = []

    # Store bounding rects for each contour that was large enough
    rects = []
    rectsUsed = []
    for cnt in large_contours:
        rects.append(cv2.boundingRect(cnt))
        rectsUsed.append(False)

    # Same for enhanced version
    e_rects = []
    e_rects_used = []
    for cnt in e_large_contours:
        e_rects.append(cv2.boundingRect(cnt))
        e_rects_used.append(False)

    # Small helper function that returns the x value
    def getXFromRect(item):
      return item[0]

    # Uses helper function to sort by x value
    rects.sort(key = getXFromRect)

    # List of rectangles that are going to be drawn at the end
    acceptedRects = []

    # Maximum horizontal distance that two bounding boxes can be in order
    # for them to be joined together into one large box
    xThr = 50

    # Loop through each bounding rect and check if it can be joined with others
    for supIdx, supVal in enumerate(rects):
      if (rectsUsed[supIdx] == False):

          # Initialize current rect
          currxMin = supVal[0]
          currxMax = supVal[0] + supVal[2]
          curryMin = supVal[1]
          curryMax = supVal[1] + supVal[3]

          # This bounding rect is used
          rectsUsed[supIdx] = True

          # Iterate all initial bounding rects
          # starting from the next
          for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

              # Initialize merge candidate
              candxMin = subVal[0]
              candxMax = subVal[0] + subVal[2]
              candyMin = subVal[1]
              candyMax = subVal[1] + subVal[3]

              # Check if x distance between current rect
              # and merge candidate is small enough
              if (candxMin <= currxMax + xThr):

                  # Reset coordinates of current rect
                  currxMax = candxMax
                  curryMin = min(curryMin, candyMin)
                  curryMax = max(curryMax, candyMax)

                  # Merge candidate (bounding rect) is used
                  rectsUsed[subIdx] = True
              else:
                  break

          # No more merge candidates possible, accept current rect
          acceptedRects.append([currxMin, curryMin, currxMax - currxMin, curryMax - curryMin])

    # Draw bounding boxes for regular frame
    for rect in acceptedRects:
      img = cv2.rectangle(frame_out, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (121, 11, 189), 2)

    # Repeat the actions for bounding boxes above for enhanced version
    e_rects.sort(key = getXFromRect)

    e_acceptedRects = []

    xThr = 50

    for supIdx, supVal in enumerate(e_rects):
      if (e_rects_used[supIdx] == False):

          # Initialize current rect
          currxMin = supVal[0]
          currxMax = supVal[0] + supVal[2]
          curryMin = supVal[1]
          curryMax = supVal[1] + supVal[3]

          # This bounding rect is used
          e_rects_used[supIdx] = True

          # Iterate all initial bounding rects
          # starting from the next
          for subIdx, subVal in enumerate(e_rects[(supIdx+1):], start = (supIdx+1)):

              # Initialize merge candidate
              candxMin = subVal[0]
              candxMax = subVal[0] + subVal[2]
              candyMin = subVal[1]
              candyMax = subVal[1] + subVal[3]

              # Check if x distance between current rect
              # and merge candidate is small enough
              if (candxMin <= currxMax + xThr):

                  # Reset coordinates of current rect
                  currxMax = candxMax
                  curryMin = min(curryMin, candyMin)
                  curryMax = max(curryMax, candyMax)

                  # Merge candidate (bounding rect) is used
                  e_rects_used[subIdx] = True
              else:
                  break

          # No more merge candidates possible, accept current rect
          e_acceptedRects.append([currxMin, curryMin, currxMax - currxMin, curryMax - curryMin])

    # Check to see if any of the accepted bounding boxes overlap with one of the
    # stationary boxes already drawn. If so, skip drawing that bounding box
    for rect in e_acceptedRects:
      x1 = rect[0]
      x2 = rect[2] + rect[0]
      y1 = rect[1]
      y2 = rect[3] + rect[1]

      valid = True
      # Loop through stationary rects
      if len(stationary_rects):
        for s_rect in stationary_rects:
          # Check for overlap
          if s_rect[0] <= rect[0] <= s_rect[2] or s_rect[0] <= rect[2] <= s_rect[2] or s_rect[1] <= rect[1] <= s_rect[3] or s_rect[1] <= rect[3] <= s_rect[3]:
            valid = False
            break
        if valid:
          # Draw box if no overlaps
          e_frame_out = cv2.rectangle(e_frame_out, (x1, y1), (x2, y2), (121, 11, 189), 2)
      else:
          # Draw box if there were no stationary boxes
        e_frame_out = cv2.rectangle(e_frame_out, (x1, y1), (x2, y2), (121, 11, 189), 2)

     # Add all the detections to the previous frame memory
      prev_store.append((enhanced_frame[y1:y2, x1:x2], [x1, y1, x2, y2]))

    # Join the regualar and enhanced frames to see side by side comparison
    joint = np.hstack([frame_out, e_frame_out])
    cv2_imshow(joint)



cap.release()

Output hidden; open in https://colab.research.google.com to view.

Object detection on daytime video without enhancement. Regular object detection on left and object detection plus stationary detection on right.

In [8]:
# Path to video
cap = cv2.VideoCapture("/content/drive/MyDrive/CS362V/data/Day_Video_With_Stops.mov")

# Object detection from Stable camera
object_detector = cv2.createBackgroundSubtractorMOG2(varThreshold=20)

# List to hold memory of previous frame for still object detection
prev_store = []

# Size of the first frame
baseline_size = 0

for i in range(80, 1000, 12):
    # Select the frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()

    if frame is None:
      break

    key = cv2.waitKey(30)
    if key == 27:
      break

    # Create an enhanced version of the frame as a copy
    enhanced_frame = frame.copy()

    # Check to see if first frame size has already been measured
    if not baseline_size:
      # Measure it if not
      baseline_size = enhanced_frame.size

    # Object detection on both the regular frame and the enhanced version
    mask = object_detector.apply(frame)
    e_mask = object_detector.apply(enhanced_frame)

    # Threshold both versions
    _, mask_thresh = cv2.threshold(mask, 180, 255, cv2.THRESH_BINARY)
    _, e_mask_thresh = cv2.threshold(e_mask, 180, 255, cv2.THRESH_BINARY)

    # Kernel for erosion
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

    # Erode both versions
    mask_eroded = cv2.morphologyEx(mask_thresh, cv2.MORPH_OPEN, kernel)
    e_mask_eroded = cv2.morphologyEx(e_mask_thresh, cv2.MORPH_OPEN, kernel)

    # Find contours in both versions
    contours, _ = cv2.findContours(mask_eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    e_contours, _ = cv2.findContours(e_mask_eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Keep only the contours that contain large objects for both versions
    min_contour_area = 500
    large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]
    e_large_contours = [cnt for cnt in e_contours if cv2.contourArea(cnt) > min_contour_area]

    # Create a copy of both versions
    frame_out = frame.copy()
    e_frame_out = enhanced_frame.copy()

    # List to track objects that were moving but stopped
    stationary_rects = []

    # Checks if previous frame detected moving objects that slowed down or stopped
    if len(prev_store):
      for prev in prev_store:
        # Get ratio of roi from previous frame size to the baseline size
        roi_to_baseline = baseline_size / prev[0].size

        # Create a grayed out roi copy of the current frame using the coordinates
        # of the roi passed in from the previous frame's detections
        gray_curr_roi = cv2.cvtColor(enhanced_frame[prev[1][1]:prev[1][3],
                                               prev[1][0]:prev[1][2]], cv2.COLOR_BGR2GRAY)

        # Create a histogram of the grayed roi
        curr_histogram = cv2.calcHist([gray_curr_roi], [0],
                         None, [256], [0, 256])

        # Create a grayed out copy of the previous frame's roi
        gray_prev_roi = cv2.cvtColor(prev[0], cv2.COLOR_BGR2GRAY)

        # Histogram of the grayed out previous roi
        prev_histogram = cv2.calcHist([gray_prev_roi], [0],
                         None, [256], [0, 256])

        # Counter to track 'difference score' between current roi and previous roi
        c1 = 0

        # Loop through each pixel in each roi and multiply the differnce in
        # histogram values by the roi to baseline ratio to normalize values
        i = 0
        while i<len(curr_histogram) and i<len(prev_histogram):
            c1+=abs((curr_histogram[i]-prev_histogram[i]) * roi_to_baseline)
            i+= 1

        # Border the roi in the current frame if the difference score is
        # less than 100000, which appears to be the range that non-motion
        # is calculated at
        if c1 < 100000:
          e_frame_out = cv2.rectangle(e_frame_out, (prev[1][0], prev[1][1]), (prev[1][2], prev[1][3]), (0, 255, 0), 2)
          stationary_rects.append(prev[1])

    # Clear previous frame memory to prepare for remembering current frame
    prev_store = []

    # Store bounding rects for each contour that was large enough
    rects = []
    rectsUsed = []
    for cnt in large_contours:
        rects.append(cv2.boundingRect(cnt))
        rectsUsed.append(False)

    # Same for enhanced version
    e_rects = []
    e_rects_used = []
    for cnt in e_large_contours:
        e_rects.append(cv2.boundingRect(cnt))
        e_rects_used.append(False)

    # Small helper function that returns the x value
    def getXFromRect(item):
      return item[0]

    # Uses helper function to sort by x value
    rects.sort(key = getXFromRect)

    # List of rectangles that are going to be drawn at the end
    acceptedRects = []

    # Maximum horizontal distance that two bounding boxes can be in order
    # for them to be joined together into one large box
    xThr = 50

    # Loop through each bounding rect and check if it can be joined with others
    for supIdx, supVal in enumerate(rects):
      if (rectsUsed[supIdx] == False):

          # Initialize current rect
          currxMin = supVal[0]
          currxMax = supVal[0] + supVal[2]
          curryMin = supVal[1]
          curryMax = supVal[1] + supVal[3]

          # This bounding rect is used
          rectsUsed[supIdx] = True

          # Iterate all initial bounding rects
          # starting from the next
          for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

              # Initialize merge candidate
              candxMin = subVal[0]
              candxMax = subVal[0] + subVal[2]
              candyMin = subVal[1]
              candyMax = subVal[1] + subVal[3]

              # Check if x distance between current rect
              # and merge candidate is small enough
              if (candxMin <= currxMax + xThr):

                  # Reset coordinates of current rect
                  currxMax = candxMax
                  curryMin = min(curryMin, candyMin)
                  curryMax = max(curryMax, candyMax)

                  # Merge candidate (bounding rect) is used
                  rectsUsed[subIdx] = True
              else:
                  break

          # No more merge candidates possible, accept current rect
          acceptedRects.append([currxMin, curryMin, currxMax - currxMin, curryMax - curryMin])

    # Draw bounding boxes for regular frame
    for rect in acceptedRects:
      img = cv2.rectangle(frame_out, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (121, 11, 189), 2)

    # Repeat the actions for bounding boxes above for enhanced version
    e_rects.sort(key = getXFromRect)

    e_acceptedRects = []

    xThr = 50

    for supIdx, supVal in enumerate(e_rects):
      if (e_rects_used[supIdx] == False):

          # Initialize current rect
          currxMin = supVal[0]
          currxMax = supVal[0] + supVal[2]
          curryMin = supVal[1]
          curryMax = supVal[1] + supVal[3]

          # This bounding rect is used
          e_rects_used[supIdx] = True

          # Iterate all initial bounding rects
          # starting from the next
          for subIdx, subVal in enumerate(e_rects[(supIdx+1):], start = (supIdx+1)):

              # Initialize merge candidate
              candxMin = subVal[0]
              candxMax = subVal[0] + subVal[2]
              candyMin = subVal[1]
              candyMax = subVal[1] + subVal[3]

              # Check if x distance between current rect
              # and merge candidate is small enough
              if (candxMin <= currxMax + xThr):

                  # Reset coordinates of current rect
                  currxMax = candxMax
                  curryMin = min(curryMin, candyMin)
                  curryMax = max(curryMax, candyMax)

                  # Merge candidate (bounding rect) is used
                  e_rects_used[subIdx] = True
              else:
                  break

          # No more merge candidates possible, accept current rect
          e_acceptedRects.append([currxMin, curryMin, currxMax - currxMin, curryMax - curryMin])

    # Check to see if any of the accepted bounding boxes overlap with one of the
    # stationary boxes already drawn. If so, skip drawing that bounding box
    for rect in e_acceptedRects:
      x1 = rect[0]
      x2 = rect[2] + rect[0]
      y1 = rect[1]
      y2 = rect[3] + rect[1]

      valid = True
      # Loop through stationary rects
      if len(stationary_rects):
        for s_rect in stationary_rects:
          # Check for overlap
          if s_rect[0] <= rect[0] <= s_rect[2] or s_rect[0] <= rect[2] <= s_rect[2] or s_rect[1] <= rect[1] <= s_rect[3] or s_rect[1] <= rect[3] <= s_rect[3]:
            valid = False
            break
        if valid:
          # Draw box if no overlaps
          e_frame_out = cv2.rectangle(e_frame_out, (x1, y1), (x2, y2), (121, 11, 189), 2)
      else:
          # Draw box if there were no stationary boxes
        e_frame_out = cv2.rectangle(e_frame_out, (x1, y1), (x2, y2), (121, 11, 189), 2)

     # Add all the detections to the previous frame memory
      prev_store.append((enhanced_frame[y1:y2, x1:x2], [x1, y1, x2, y2]))

    # Join the regualar and enhanced frames to see side by side comparison
    joint = np.hstack([frame_out, e_frame_out])
    cv2_imshow(joint)



cap.release()

Output hidden; open in https://colab.research.google.com to view.